In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

In [2]:
import mlflow
import os
import boto3 

# PROFILE = os.getenv("AWS_PROFILE")
# ACCESS_ID = os.getenv("AWS_ACCESS_KEY_ID")
# ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

# session = boto3.Session(profile_name=PROFILE)
# s3_client = session.client('s3',
#             aws_access_key_id=ACCESS_ID,
#             aws_secret_access_key=ACCESS_KEY)

TRACKING_URI = "ec2-52-53-237-125.us-west-1.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_URI}:5000")
mlflow.set_experiment("taxi_trip_prediction-experiment")

<Experiment: artifact_location='s3://mlflow-artifacts-prem/1', experiment_id='1', lifecycle_stage='active', name='taxi_trip_prediction-experiment', tags={}>

In [3]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df
    
def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [4]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [5]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/Users/premsurawut/anaconda3/envs/course-mlops-zoomcamp/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'ec2-52-53-237-125.us-west-1.compute.amazonaws.com'
RUN_ID = 'e1ddb186047749a08a0d6462056cfa99'

client = MlflowClient(tracking_uri=f"http://{MLFLOW_TRACKING_URI}:5000")

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv